# 基本的图算法

In [2]:
import Queue

class Node:
    def __init__(self, value=None):
        self.value = value
        # color is "W" for white, "G" for gray, "B" for black
        self.color = "W"
        #depth means the length to root in breadth_first_search
        #and find time in depth_first_search
        self.depth = None
        self.final = None #final time used in depth_first_search
        self.pre = None
    def reset(self):
        self.color = "W"
        self.depth = None
        self.final = None #final time used in depth_first_search
        self.pre = None
        
#通过领接链表保存图
class Graphy:
    def __init__(self):
        self.node_neighbors = {} #map list
        self.visited = {}
        self.time = 0
    
    def add_nodes(self, nodelist):
        for node in nodelist:
            self.add_node(node)
    
    def add_node(self, node):
        if not node in self.nodes():
            self.node_neighbors[node] = []
    
    def add_edge(self, edge, bDi=True):
        u,v = edge
        if(v not in self.node_neighbors[u]):
            self.node_neighbors[u].append(v)
        if(False == bDi):
            if(u not in self.node_neighbors[v]):
                self.node_neighbors[v].append(u)

    def nodes(self):
        return self.node_neighbors.keys()
    
    #有向图的转置
    def transposition_graphy(self):
        g = Graphy()
        g.add_nodes(self.node_neighbors.keys())
        for key in self.node_neighbors.keys():
            for node in self.node_neighbors[key]:
                g.add_edge((node,key))
        return g
    
    #广度优先遍历
    def breadth_first_search(self, s):
        for node in self.node_neighbors.keys():
            node.color = "W"
            node.depth = None
            node.pre = None
        s.color = "G"
        s.depth = 0
        s.pre = None
        Q = Queue.Queue()
        Q.put(s)
        while(not Q.empty()):
            u = Q.get()
            for v in self.node_neighbors[u]:
                if("W" == v.color):
                    v.color = "G"
                    v.depth = u.depth+1
                    v.pre = u
                    Q.put(v)
            u.color = "B"

    #深度优先遍历
    def depth_first_search(self):
        for node in self.node_neighbors.keys():
            node.color = "W"
            node.depth = None
            node.final = None
            node.pre = None
        self.time = 0
        for node in self.node_neighbors.keys():
            if("W" == node.color):
                self.dfs_visit(node)
    def dfs_visit(self, u):
        self.time += 1
        u.depth = self.time
        u.color = "G"
        for node in self.node_neighbors[u]:
            if("W" == node.color):
                node.pre = u
                self.dfs_visit(node)
        u.color = "B"
        self.time += 1
        u.final = self.time
        
    #拓扑排序    
    def topological_sort(self, isSort=True):
        if(True == isSort):
            self.depth_first_search()
        nodelist = []
        insert_pos = 0
        for node in self.node_neighbors.keys():
            for insert_pos in range(0,len(nodelist)+1):
                if(insert_pos >= len(nodelist)):
                    break;
                if(node.final > nodelist[insert_pos].final):
                    break
            nodelist.insert(insert_pos, node)
        return nodelist
    
    def strongly_connected_components(self):
        self.depth_first_search()
        node_list = self.nodes()
        g = self.transposition_graphy()
        nodelist = g.topological_sort(False)
        tree = []
        g.time = 0
        for node in nodelist:
            node.reset()
        for node in nodelist:
            if("W" == node.color):
                g.dfs_visit(node)
                tree.append(node)
        return tree
        
    
g = Graphy()
nodes = [Node(i) for i in range(8)]
g.add_nodes(nodes)
g.add_edge((nodes[0], nodes[1]))
g.add_edge((nodes[1], nodes[2]))
g.add_edge((nodes[1], nodes[5]))
g.add_edge((nodes[1], nodes[4]))
g.add_edge((nodes[2], nodes[3]))
g.add_edge((nodes[2], nodes[6]))
g.add_edge((nodes[3], nodes[7]))
g.add_edge((nodes[3], nodes[2]))
g.add_edge((nodes[4], nodes[0]))
g.add_edge((nodes[4], nodes[5]))
g.add_edge((nodes[5], nodes[6]))
g.add_edge((nodes[6], nodes[5]))
g.add_edge((nodes[6], nodes[7]))

print("breadth first search:")
g.breadth_first_search(nodes[0])
for node in g.nodes():
    print (node.value, node.color, node.depth)

print("depth first search:")
g.depth_first_search()
node_list = g.topological_sort()
for node in node_list:
    print (node.value, node.color, node.depth, node.final)

print("nums of strongly connected components:")
treelist = g.strongly_connected_components()
print(len(treelist))


breadth first search:
(2, 'B', 2)
(4, 'B', 2)
(5, 'B', 2)
(6, 'B', 3)
(7, 'B', 4)
(0, 'B', 0)
(3, 'B', 3)
(1, 'B', 1)
depth first search:
(4, 'B', 11, 16)
(0, 'B', 12, 15)
(1, 'B', 13, 14)
(2, 'B', 1, 10)
(6, 'B', 6, 9)
(5, 'B', 7, 8)
(3, 'B', 2, 5)
(7, 'B', 3, 4)
nums of strongly connected components:
4


# 最小生成树

In [9]:
import numpy as np

# 定义顶点
class Vertex:
    def __init__(self, key):
        self.id = key
        self.adj={}
        self.parent=None
        self.rank=0
    def addNeighbor(self,nbr,weight=0):
        self.adj[nbr]=weight
    def getNeighbors(self):
        # return an array of neighbors
        return self.adj.keys()
    def getId(self):
        return self.id
    def getWeight(self,key):
        return self.adj[key]

class Edge(object):
    def __init__(self,u,v,weight):
        self.u=u
        self.v=v
        self.weight=weight

def Find(v):
    if(None == v.parent):
        return v
    else:
        v.parent=Find(v.parent)
        return v.parent
        
def Union(u,v):
    if(u.rank <= v.rank):
        u.parent = v
        if u.rank==v.rank:
            v.rank+=1
    else:
        v.parent = u

class KruskalQueue:
    def __init__(self, elist):
        self.elist = elist
        self.size=len(self.elist)
    def build_heap(self):
        # 此函数按权值使用堆排序法降序排列self.elist
        
        # xrange 用法与 range 完全相同，所不同的是生成的不是一个list对象，而是一个生成器。
        # xrange 不用开辟一块很大的空间，所以速度比list快很多
        for i in xrange(self.size/2-1,-1,-1):
            self.perDown(i)
    def del_min(self):
        self.elist[0],self.elist[-1]=self.elist[-1],self.elist[0]
        e = self.elist.pop()
        self.size -= 1
        self.per_down(0)
        return e
    def per_down(self, i):
        # 按权值降序排列self.elist
        left=2*i+1
        right=2*i+2
        little=i
        if(left<=self.size-1 and self.elist[i].weight>self.elist[left].weight):
            little=left
        if(right<=self.size-1 and self.elist[little].weight>self.elist[right].weight):
            little=right
        if(little!=i):
            self.elist[i],self.elist[little]=self.elist[little],self.elist[i]
            self.perDown(little)
    def per_up(self, i):
        if(i>0 and self.elist[i].weight<self.elist[(i-1)/2].weight):
            self.elist[i],self.elist[(i-1)/2]=self.elist[(i-1)/2],self.elist[i]
            self.perUp((i-1)/2)
        
def Kruskal(G):
    elist=[]
    accepted_e_list=[]
    for v in G:
        for vertex in v.getNeighbors():
            e=Edge(v,vertex,v.getWeight(vertex))
            elist.append(e)
    queue=KruskalQueue(elist)
    queue.build_heap()
    edge_num = 0
    while(edge_num < G.size-1):
        e = queue.del_min()
        u = e.u
        v = e.v
        uset = Find(u)
        vset = Find(v)
        if(uset != vset):
            accepted_e_list.append(e)
            edge_num+=1
            Union(uset, vset)
    return accepted_e_list


class MSTree:
    def __init__(self):
        self.root = None

class MSTNode:
    def __init__(self, key):
        self.id = key
        self.parent = None
        self.rank = 0
        
# 通过领接矩阵保存图
class UndiGraph:
    def __init__(self):
        self.nodes = []
        self.matrix = None
        self.edge_list = []
    
    def create_matrix(self, nodes):
        self.nodes = nodes
        len(nodes)
        self.matrix = np.zeros([len(nodes), len(nodes)], dtype = np.int16)
    
    # 无向图
    def add_edge(self, edge):
        u,v,weight = edge
        if(u == v):
            return
        if((u in self.nodes) and (v in self.nodes)):
            self.edge_list.append((MSTNode(u), MSTNode(v),weight))
            row = self.nodes.index(u)
            col = self.nodes.index(v)
            self.matrix[row, col] = weight
            self.matrix[col, row] = weight
    
    def sort_edge_list(self):
        def cmp(e1,e2):
            if(e1[2] < e2[2]):
                return -1
            elif(e1[2] > e2[2]):
                return 1
            else:
                return 0
        self.edge_list.sort(cmp)
    
    def mst_kruskal(self):
        self.sort_edge_list()
        accpeted_e_list=[]
        for edge in self.edge_list:
            u = edge[0]
            v = edge[1]
            uset=Find(u)
            vset=Find(v)
        if(uset!=vset):
            accpeted_e_list.append((u.id,v.id))
            Union(uset,vset)
        return accpeted_e_list

if __name__=='__main__':
    nodes = ['a','b','c','d','e','f','g','h','i']
    ug = UndiGraph()
    ug.create_matrix(nodes)
    ug.add_edge(('a','b',4))
    ug.add_edge(('a','h',8))
    ug.add_edge(('b','c',8))
    ug.add_edge(('b','h',11))
    ug.add_edge(('c','d',7))
    ug.add_edge(('c','f',4))
    ug.add_edge(('c','i',2))
    ug.add_edge(('d','e',9))
    ug.add_edge(('d','f',14))
    ug.add_edge(('e','f',10))
    ug.add_edge(('f','g',2))
    ug.add_edge(('g','h',1))
    ug.add_edge(('g','i',6))
    ug.add_edge(('h','i',7))
    
    arr = ug.mst_kruskal()
    print(arr)
    #print(ug.edge_list)


[('d', 'f')]
